Last Update @ 2020.12.04

- Huggingface Transformers 4.0.0  버전 반영

# Package 설치 & 데이터 받기

In [ ]:
# %%bash
# apt-get update
# apt-get install g++ openjdk-8-jdk python-dev python3-dev
# pip3 install JPype1
# pip3 install konlpy

In [ ]:
# %env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [ ]:
# %%bash
# bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
# pip3 install /tmp/mecab-python-0.996

In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
my_path = '/content/notebooks'
save_path = '/content/drive/MyDrive/Colab Notebooks/package' ## 패키지가 저장될 경로

os.symlink(save_path, my_path)
sys.path.insert(0, my_path)

In [ ]:
# !pip install --target=$my_path mxnet # 이때, 원하는 경로에 설치하기 위해 --target 옵션을 지정해줍니다.

!pip install --target=$my_path mxnet
!pip install --target=$my_path gluonnlp pandas tqdm
!pip install --target=$my_path sentencepiece
!pip install --target=$my_pathtransformers

In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
my_path = '/content/notebooks'
save_path = '/content/drive/MyDrive/Colab Notebooks/package' 

os.symlink(save_path, my_path)
sys.path.insert(0, my_path)

In [ ]:
from transformers import BertTokenizerFast, AlbertModel, BertModel, AutoTokenizer

In [ ]:
!pip install hanja
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install  transformers
!pip install torch
!pip install hanja
!pip install imblearn
!pip install WordCloud
!apt-get install fonts-nanum*
!apt-get install fontconfig

In [ ]:
from transformers import BertTokenizerFast, AlbertModel, BertModel, AutoTokenizer
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

import hanja
from imblearn.over_sampling import RandomOverSampler

from transformers.optimization import get_cosine_schedule_with_warmup, get_linear_schedule_with_warmup
from transformers import AdamW

from collections import defaultdict
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/temp/train_data.csv', encoding='cp949')
data.title=data.title.astype(str)

val_dataset = pd.read_csv('/content/drive/MyDrive/temp/valid.csv', encoding='cp949')
val_dataset.title=val_dataset.title.astype(str)

In [ ]:
X_train, X_val, Y_label1_train, Y_label1_val =  train_test_split(data, test_size = 0.2, random_state = 42)

In [ ]:
# 뉴스 데이터 클래스 생성
class NewsSubjectDataset(Dataset):
  def __init__(self, subjects, targets, tokenizer, max_len):
    self.subjects = subjects
    self.targets = targets
    self.tokenizer = tokenizer
    self.max_len = max_len
  def __len__(self):
    return len(self.subjects)
  def __getitem__(self, item):
    subject = str(self.subjects[item])
    target = self.targets[item]
    encoding = self.tokenizer.encode_plus(
      subject,
      add_special_tokens=True,
      max_length=self.max_len,
      return_token_type_ids=False,
      padding = 'max_length',
      truncation = True,
      return_attention_mask=True,
      return_tensors='pt',
    )
    return {
      'subject_text': subject,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'targets': torch.tensor(target, dtype=torch.long)
    }
def create_data_loader(df, tokenizer, max_len, batch_size, shuffle_=False, valid=False):
  if valid:
    ds = NewsSubjectDataset(
      subjects=df.title.to_numpy(),
      targets=np.zeros(len(df)),
      tokenizer=tokenizer,
      max_len=max_len
      )
  else:
    ds = NewsSubjectDataset(
      subjects=df.title.to_numpy(),
      targets=df.topic_idx.to_numpy(),
      tokenizer=tokenizer,
      max_len=max_len
    )
  return DataLoader(
    ds,
    batch_size=batch_size,
    num_workers=4,
    shuffle = shuffle_
  )

In [ ]:
# 데이터 로더 생성
BATCH_SIZE = 128
MAX_LEN = 16

In [ ]:
titles_t = data.title.to_numpy().reshape(-1,1)
labels_t = data.topic_idx.to_numpy().reshape(-1,1)
# oversample = RandomOverSampler()
# X_over, y_over = oversample.fit_resample(titles_t, labels_t)
train = pd.DataFrame({'title':titles_t.reshape(-1), 'topic_idx':labels_t.reshape(-1)})
tokenizer_kcbert = AutoTokenizer.from_pretrained("beomi/kcbert-large")
train_data_loader = create_data_loader(train, tokenizer_kcbert, MAX_LEN, BATCH_SIZE, shuffle_=True)
valid_data_loader = create_data_loader(val_dataset, tokenizer_kcbert, MAX_LEN, BATCH_SIZE, valid=True)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
# def remove_and_substitute(title):
#   x = title[:]
#   x = re.sub('\.\.\.', '', x)
#   x = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z\s]', ' ', x)
#   x = hanja.translate(x, 'substitution')
#   return x.strip()
  
# k = train['title'].apply(remove_and_substitute)
# train.title = k
# k = df_valid['title'].apply(remove_and_substitute)
# df_valid.title = k

In [ ]:
class NewsSubjectClassifier_kcbert(nn.Module):
  def __init__(self, n_classes):
    super(NewsSubjectClassifier_kcbert, self).__init__()
    self.bert = BertModel.from_pretrained("beomi/kcbert-large")
    self.drop = nn.Dropout(p=0.5)
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
  def forward(self, input_ids, attention_mask):
    _, pooled_output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask,
       return_dict=False
    )
    output = self.drop(pooled_output)
    return self.out(output)

In [ ]:
device = torch.device("cuda")
EPOCHS = 10
loss_fn = nn.CrossEntropyLoss().to(device)

In [ ]:
def train_epoch(model,data_loader,loss_fn,optimizer,device,scheduler,n_examples):
  model = model.train()
  losses = []
  correct_predictions = 0
  for ii, d in tqdm(enumerate(data_loader)):
    input_ids = d["input_ids"].to(device)
    attention_mask = d["attention_mask"].to(device)
    targets = d["targets"].to(device)
    outputs = model(
      input_ids=input_ids,
      attention_mask=attention_mask
    )
    _, preds = torch.max(outputs, dim=1)
    loss = loss_fn(outputs, targets)
    correct_predictions += torch.sum(preds == targets)
    losses.append(loss.item())
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()
    if ii % 100 == 2 :
        print(f'Train accuracy {correct_predictions.double() / len(train)} , loss {np.mean(losses)}', ii, "th")

  return correct_predictions.double() / n_examples, np.mean(losses) #train_acc, train_loss

In [ ]:
def get_predictions(model, data_loader):
  model = model.eval()
  subject_texts = []
  predictions = []
  prediction_probs = []
  with torch.no_grad():
    for ii, d in enumerate(data_loader):
      texts = d["subject_text"]
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["targets"].to(device)
      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      _, preds = torch.max(outputs, dim=1)
      subject_texts.extend(texts)
      predictions.extend(preds)
      prediction_probs.extend(outputs)
  predictions = torch.stack(predictions).cpu()
  prediction_probs = torch.stack(prediction_probs).cpu()
  return subject_texts, predictions, prediction_probs
import gc

In [ ]:
model_kcbert = NewsSubjectClassifier_kcbert(n_classes=19).to(device)
optimizer = AdamW(model_kcbert.parameters(), lr=1e-4)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=int(total_steps*0.1), num_training_steps=total_steps)

Some weights of the model checkpoint at beomi/kcbert-large were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is de

In [ ]:
EPOCHS = 5

In [ ]:
for epoch in range(EPOCHS):
  print(f'Epoch {epoch + 1}/{EPOCHS}')
  print('-' * 10)
  train_acc, train_loss = train_epoch(
    model_kcbert,
    train_data_loader,
    loss_fn,
    optimizer,
    device,
    scheduler,
    len(train)
  )
  print(f'Train loss {train_loss} accuracy {train_acc}')

In [ ]:
model2 = model_kcbert.eval()
subject_texts = []
predictions = []
prediction_probs = []
with torch.no_grad():
    for ii, d in tqdm(enumerate(valid_data_loader)):
        texts = d["subject_text"]
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        targets = d["targets"].to(device)
        outputs = model2(
        input_ids=input_ids,
        attention_mask=attention_mask
        )
        _, preds = torch.max(outputs, dim=1)
        subject_texts.extend(texts)
        predictions.extend(preds)
        prediction_probs.extend(outputs)
predictions = torch.stack(predictions).cpu()
prediction_probs = torch.stack(prediction_probs).cpu()

In [ ]:
# >>> a = torch.randn(4, 4)
# >>> a
# tensor([[-1.2360, -0.2942, -0.1222,  0.8475],
#         [ 1.1949, -1.1127, -2.2379, -0.6702],
#         [ 1.5717, -0.9207,  0.1297, -1.8768],
#         [-0.6172,  1.0036, -0.6060, -0.2432]])
# >>> torch.max(a, 1)
# torch.return_types.max(values=tensor([0.8475, 1.1949, 1.5717, 1.0036]), indices=tensor([3, 0, 0, 1]))

In [ ]:
answers = []

y_review_texts, y_pred, y_pred_probs = get_predictions(
  model_kcbert,
  valid_data_loader
)

answers.append(y_pred.tolist())